In [1]:
import pandas as pd
import numpy as np

In [2]:
def normalized_value_counts(series):
   counts = series.value_counts(sort=True)
   percentages = series.value_counts(normalize=True, sort=True).mul(100).round(2).astype(str) + '%'
   result = pd.concat([counts, percentages], axis=1, keys=['N', '%'])
   
   return result

In [15]:
codebook = pd.read_excel('codebook.xlsx')
baseline = pd.read_csv('tba_data.csv')

In [18]:
# codebook

In [16]:
baseline

,enumerator_name,location,gender,age_range,qual_life1,qual_life8,qual_life9,qual_life12,qual_life13,qual_life14,...,rapport3,partner_feedback,security1,security2,recognition,justice,recognition_justice,dignity,emergency_irm,comments
0,sb,gaziantep,male,25-55,good,not_at_all,moderate_amount,completely,a_little,a_little,...,yes,تشكر الجهود فقط,no,NaN,very_much,very_much,very_much,very_much,نقلة المنزل,لا
1,sb,gaziantep,male,25-55,good,not_at_all,moderate_amount,moderately,mostly,a_little,...,yes,كانو متعاونين إلى حد كبير,no,NaN,very_much,a_little,a_little,somewhat,صرف على أجور المعيشة والتنقل وقت الزلزال,يفضل ان يتم إعطاء الدعم المناسب لكل حالة حسب و...
2,sb,gaziantep,male,25-55,neither,moderate_amount,very_much,not_at_all,mostly,not_at_all,...,yes,مسرور من المشاركين بالمشروع,no,NaN,totally,totally,very_much,very_much,اجار منزل في اسطنبول,لا
3,sb,gaziantep,male,25-55,neither,not_at_all,very_much,a_little,moderately,a_little,...,yes,لا كل الشكر,no,NaN,very_much,very_much,totally,very_much,اجار للمنزل,يفضل ان يسرع المشروع بسبب الحاجة للدعم المادي
4,sb,gaziantep,female,over_55,very_poor,not_at_all,a_little,a_little,a_little,moderately,...,yes,مشكورين,no,NaN,very_much,very_much,very_much,somewhat,ساعدت فيه ابنتي ودفعت اجار البيت والفواتير,اتمنى ان يتم اضافة المساعدات الطبية من حيث الت...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,bh,urfa,male,25-55,good,extreme_amount,a_little,moderately,a_little,completely,...,somewhat,NaN,no,NaN,somewhat,not_at_all,not_at_all,not_at_all,NaN,NaN
102,bh,urfa,male,25-55,poor,not_at_all,a_little,moderately,a_little,completely,...,yes,NaN,no,NaN,totally,totally,totally,totally,المستلزمات الأساسية خلال فترة الزلزال التي خرج...,اتمنى ان يكون المبلغ جيد بما يكفي احتياجي لإجر...
103,bh,urfa,male,over_55,neither,not_at_all,a_little,not_at_all,mostly,not_at_all,...,yes,NaN,yes,اتخوف من تسرب المعلومات الشخصية للمشاركين وذلك...,somewhat,a_little,totally,somewhat,NaN,انا كفيل للأيتام وأرجو مساعدتي بتأمين أجور تدر...
104,bh,urfa,female,25-55,neither,a_little,moderate_amount,a_little,a_little,moderately,...,yes,NaN,yes,لا أعلم,very_much,somewhat,very_much,very_much,NaN,NaN


In [19]:
baseline['enumerator_name'].value_counts()

enumerator_name
mm    33
bh    30
sb    27
nk    16
Name: count, dtype: int64

In [20]:
data = baseline['qual_life1']
normalized_value_counts(series=data)

,N,%
qual_life1,,
neither,32,30.19%
poor,28,26.42%
good,25,23.58%
very_poor,17,16.04%
very_good,4,3.77%


In [23]:
data = baseline['qual_life2']
normalized_value_counts(series=data)

,N,%
qual_life2,,
satisfied,41,38.68%
dissatisfied,27,25.47%
neither,25,23.58%
very_dissatisfied,7,6.6%
very_satisfied,6,5.66%


In [26]:
data = baseline['qual_life3']
normalized_value_counts(series=data)

,N,%
qual_life3,,
moderate_amount,32,30.19%
a_little,24,22.64%
very_much,22,20.75%
not_at_all,17,16.04%
extreme_amount,11,10.38%
